<a href="https://colab.research.google.com/github/crodier1/data_science/blob/main/PyTorch_CNN_Facial_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install opendatasets

import opendatasets as od

od.download("https://www.kaggle.com/competitions/challenges-in-representation-learning-facial-expression-recognition-challenge/data")

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from torch.utils.data import Dataset
from datetime import datetime
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv('challenges-in-representation-learning-facial-expression-recognition-challenge/train.csv')

Y = df['emotion']
X = df['pixels']

FileNotFoundError: [Errno 2] No such file or directory: 'challenges-in-representation-learning-facial-expression-recognition-challenge/train.csv'

In [ ]:
def to_array(x):
  return np.array([int(i) / 255 for i in str(x).split(' ')]).reshape(1,48,48)

X = X.apply(lambda x: to_array(x))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,Y,
                                   random_state=104,
                                   test_size=0.25,
                                   shuffle=True)

In [ ]:
class FaceDataSet(Dataset):
  def __init__(self, x, y):
    self.x = x
    self.y = y

  def __len__(self):
    return len(self.x)

  def __getitem__(self, index):
    image = self.x.iloc[index]
    image = torch.Tensor(image)
    y_label = torch.tensor(self.y.iloc[index])


    return image,y_label

In [ ]:
train_data_set = FaceDataSet(X_train, y_train)
test_data_set = FaceDataSet(X_test, y_test)

In [ ]:
K = len(set(Y))
K

In [ ]:
batch_size = 128

train_loader = torch.utils.data.DataLoader(
    dataset= train_data_set,
    batch_size = batch_size,
    shuffle = True
)

print(train_loader.dataset[0][0].shape)

test_loader = torch.utils.data.DataLoader(
    dataset= test_data_set,
    batch_size = batch_size
)

In [ ]:
class CNN(nn.Module):
  def __init__(self, K):
    super(CNN, self).__init__()
    self.conv_layers = nn.Sequential(
      nn.Conv2d(in_channels = 1, out_channels = 32, kernel_size = 3, stride=2),
      nn.ReLU(),
      nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size = 3, stride=2),
      nn.ReLU(),
      nn.Conv2d(in_channels = 64, out_channels = 128, kernel_size = 3, stride=2),
      nn.ReLU()
      )

    self.dense_layers = nn.Sequential(
        nn.Dropout(0.2),
        nn.Linear(128 * 2 * 2, 512),
        nn.ReLU(),
        nn.Dropout(0.2),
        nn.Linear(512, K)
    )
  def forward(self, X):
    out = self.conv_layers(X)
    out = out.view(out.size(0), -1)
    out = self.dense_layers(out)
    return out


# class CNN(nn.Module):
#   def __init__(self, K):
#     super(CNN, self).__init__()

#     self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=2)
#     self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=2)
#     self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=2)

#     self.fc1 = nn.Linear(16, 1024)
#     self.fc2 = nn.Linear(1024, K)

#   def forward(self, x):
#     x = F.relu(self.conv1(x))
#     x = F.relu(self.conv2(x))
#     x = F.relu(self.conv3(x))
#     print(x.shape)
#     x = x.view(x.size(0), -1)
#     x = F.dropout(x, p=0.5)
#     x = F.relu(self.fc1(x))
#     x = F.dropout(x, p=0.2)
#     x = self.fc2(x)
#     return x

In [ ]:
model = CNN(K)

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

In [ ]:
def batch_gd(model, criterion, optimizer, train_loader, test_loader, epochs):
  train_losses = np.zeros(epochs)
  test_losses = np.zeros(epochs)

  for it in range(epochs):
    model.train()
    t0 = datetime.now()
    train_loss = []

    for inputs, targets in train_loader:

      inputs, targets = inputs.to(device), targets.to(device)

      optimizer.zero_grad()

      outputs = model(inputs)
      loss = criterion(outputs, targets)

      loss.backward()
      optimizer.step()

      train_loss.append(loss.item())

    train_loss = np.mean(train_loss)

    model.eval()

    test_loss = []

    for inputs, targets in test_loader:
      inputs, targets = inputs.to(device),  targets.to(device)

      outputs = model(inputs)

      loss = criterion(outputs, targets)

      test_loss.append(loss.item())

    test_loss = np.mean(test_loss)

    train_losses[it] = train_loss
    test_losses[it] = test_loss

    dt = datetime.now() - t0


    print(f'Epoch {it+1}/{epochs}, Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}, Duration: {dt}')


  return train_losses, test_losses

In [ ]:
train_losses, test_losses = batch_gd(model, criterion, optimizer, train_loader, test_loader, 15)